In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [25]:
# 크롬 옵션 객체 생성
options = Options()

#지정한 user-agent로 설정합니다.
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36")

options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')

In [31]:
driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options)

/var/folders/9t/s4kfwbj54wj1sw01v7vtb68h0000gn/T/ipykernel_55861/4032167621.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options)


In [26]:
driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options)
url_list = []
content_list = " "
text = input('어떤 키워드로 검색하시겠습니까? : ')
 
for i in range(1, 100):  # 1~n페이지까지의 블로그 내용을 읽어옴
    url = 'https://section.blog.naver.com/Search/Post.nhn?pageNo='+ str(i) + '&rangeType=ALL&orderBy=sim&keyword=' + text
    driver.get(url)
    time.sleep(1)
    
    for j in range(1, 7): # 각 블로그 주소 저장
        titles = driver.find_element(By.XPATH,'/html/body/ui-view/div/main/div/div/section/div[2]/div['+str(j)+']/div/div[1]/div[1]/a[1]')
        title = titles.get_attribute('href')
        url_list.append(title)
print("url 수집 끝, 해당 url 데이터 크롤링")

/var/folders/9t/s4kfwbj54wj1sw01v7vtb68h0000gn/T/ipykernel_55861/101640373.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options)


어떤 키워드로 검색하시겠습니까? : 시각장애인
url 수집 끝, 해당 url 데이터 크롤링


In [27]:
len(url_list)

594

In [ ]:
Lists = []
for url in url_list: # 수집한 url 만큼 반복
    driver.get(url) # 해당 url로 이동
    driver.switch_to.frame('mainFrame')
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    
    overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
    contents = driver.find_elements(By.CSS_SELECTOR,(overlays))
    names = soup.title.text.replace(": 네이버 블로그", "").replace(": 네이버블로그", "")
    try :
        dates = soup.select_one('span.se_publishDate').text.replace('. ', '/')
    except :
        dates = soup.select_one('p.date').text.replace('. ', '/')
    for content in contents:
        content_list = content_list + content.text.replace('\n', ' ') # content_list 라는 값에 + 하면서 점점 누적
    Lists.append([dates, names, content_list])
    content_list = ' '

driver.quit()

In [33]:
df = pd.DataFrame(data= Lists, columns=['포스팅 날짜', '글 제목', '본문'])
df

,포스팅 날짜,글 제목,본문
0,23시간 전,서로를 알아가는 법 (ft. 시각장애인 에티켓),"지난 낭독 동행 여행 사전 미팅 이후, 많은 가르침을 주셨던 김 선생님과 매일 톡..."
1,6시간 전,"한 치 앞 안 보인 학교, 시각 장애인 총장이 정상화","한 치 앞 안 보인 학교, 시각 장애인 총장이 정상화 이재서 총신대 前총장, 임기..."
2,2023/3/15/16:22,시각장애인 대면 낭독봉사 후기,vms에서 새롭게 할 수 있는 봉사활동을 찾던 도중 생소한 봉사활동을 발견했다. ...
3,2023/3/27/23:05,안마바우처 신청 시각장애인 마사지 받아요~,안마바우처로 시각장애인 마사지 받아요저는 안마 받는 걸 너무나 좋아합니다 오늘처럼...
4,2023/5/12/10:10,(강의 리뷰) 11) 울산광역시 시각장애인 주간보호 센터 (배달강좌) 과정,#울산남구청#평생교육과#배달강좌#울산시각장애인주간보호센터#건강강좌#천샘의건강교실#...
...,...,...,...
402,2023/5/18/9:30,장애인인식개선교육 신청 [시각장애인],WayMakers 말 그대로 길을 만드는 사람들. 저희 에스엠플래닛에서는 시각장애...
403,2023/4/26/15:05,[시각장애인 각막이식 수술지원] - 청주시 시각장애인의 희망의 빛을 되찾길 바라며,"청주시 시각장애인에게 각막이식 수술을 지원하기 위해 충북대학교병원, 상당라이온스클..."
404,2010/11/15/9:40,굿 디자인_시각장애인을 위한 도우미식판 제품디자인,
405,2023/3/21/10:41,귀로 듣는 쇼핑 할 수 있을까? 시각장애인을 위한 쇼핑 플랫폼 '소리마켓',오늘은 시각장애인을 위한 쇼핑 플랫폼 '소리마켓'을 소개하려고 해요. 시각장애인이...


In [38]:
df.to_excel(f'블로그_{text}.xlsx', index = False)